# Test Matching Insee/PM

## INSEE

- https://s3.console.aws.amazon.com/s3/object/calfdata/INSEE/Stock/ETS/
        - INSEE/Stock/ETS/StockEtablissement_utf8.csv
        
```
['siren', 'siret']
```

## INPI

- https://s3.console.aws.amazon.com/s3/buckets/calfdata/INPI/TC_1/Stock_processed/
    - INPI/TC_1/Stock_processed/initial_PM.gz
    - INPI/TC_1/Stock_processed/initial_PM.json
    
Colonnes test:

```
["Siren","Date_Immatriculation", "Date_Clôture", "Date_Greffe"]
```

## Sauvegarde

* La liste des SIREN matchés sera sauvegardée selon leur nature et origine
  * nature → ACTES/COMPTES/ETS/etc
  * origine → initial/partiel/new/evt

Les matchés seront sauvegardé dans calfdata/SIRETISATION/matche/ au format suivant:

* insee_nature_origine_matche.gz
    * ex: insee_pm_initial_matche.gz
    
    

## Moteur de recherche TEST

* Insee
  * http://avis-situation-sirene.insee.fr/IdentificationListeSiret.action
* INPI/TC
  * https://data.inpi.fr/


In [1]:
import boto3, json
import dask.dataframe as dd
import pandas as pd
import Match_inpi_insee.aws_connectors as aws
#from tqdm.notebook import tqdm
#import tqdm
%load_ext autoreload
%autoreload 2

In [2]:
instance_aws = 'https://calfdata.s3.eu-west-3.amazonaws.com'
bucket = 'calfdata'

AWS_connection = aws.aws_instantiate(instance_aws, bucket)
AWS_connection.uploadFileBucket(pathfile ="INPI/TC_1/Stock_processed/initial_PM.json")
with open('initial_PM.json', 'r') as f:
    pm_json = json.load(f)

In [3]:
insee = AWS_connection.url_instance_bucket(path_file = "INSEE/Stock/ETS/StockEtablissement_utf8.csv")
pm = AWS_connection.url_instance_bucket(path_file = "INPI/TC_1/Stock_processed/initial_PM.gz")

In [4]:
data_ets_= dd.read_csv(insee, 
                       usecols = ['siren', 'siret'],
                       dtype={'siren':'object',
                              'siret':'object'}
                      )
data_pm_= dd.read_csv(pm, 
                       usecols =["Siren",
                                 #"Date_Immatriculation",
                                 #"Date_Clôture", "Date_Greffe"
                                ],
                      dtype={'Siren':'object'},
                      compression='gzip',
                      blocksize=None,
                      low_memory=False
                      )

In [ ]:
test = data_ets_.merge(
    data_pm_,
    how= 'right',
    left_on = "siren", 
    right_on ='Siren',
    indicator = True).compute()

In [ ]:
test.head()

In [ ]:
test.groupby('_merge')['_merge'].count()

In [ ]:
test.loc[lambda x: x['_merge'].isin(['right_only'])].head()

In [ ]:
test.loc[lambda x: x['siren'].isin(['007350168'])]

In [ ]:
len('007350168')

In [ ]:
#pm_stock_initial = AWS_connection.load_df_dd(
#    path_file = "INPI/TC_1/Stock_processed/initial_PM.gz",
#                          dtypes = None,
#                          usecols = None)

In [ ]:
#insee_etb = AWS_connection.load_insee(option ='etb', for_matching = True)

In [ ]:
#insee_etb.head()